In [36]:
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import jieba
import nltk
from collections import OrderedDict
%matplotlib inline

In [2]:
risk_qa = pd.read_excel("risk_data.xlsx")

In [3]:
risk_qa.head()

,客戶編號,風險等級,風險等級分類,適性評量填寫日期,適性版號,題目序號,KYC_QUESTION,ANS_ORDER,ANS_ITEM,答題得分,客戶職業與任職部門,AGENT代碼
0,261909,5,積極型,2018-10-30 22:54:34.523,201809A,A0,是否為其他國家稅務居住者?,1,1.否,0,NaN,9999999999
1,261909,5,積極型,2018-10-30 22:54:34.523,201809A,A01,職業／行業別,8,8.退休人士,0,NaN,9999999999
2,261909,5,積極型,2018-10-30 22:54:34.523,201809A,A02,職務,1,1.基層職員,0,NaN,9999999999
3,261909,5,積極型,2018-10-30 22:54:34.523,201809A,A03,任職單位,1,1.任職公司／機構,0,寶豐,9999999999
4,261909,5,積極型,2018-10-30 22:54:34.523,201809A,A03,任職單位,2,2.任職部門,0,營建,9999999999


In [4]:
uni_id = np.unique(risk_qa["客戶編號"])

In [5]:
attr_dict = defaultdict(lambda: defaultdict(list))
max_ans_dict = defaultdict(int) # record max answer num for each question (for one-hot array)

for index, row in risk_qa.iterrows():
    attr_dict[row["客戶編號"]][row["題目序號"]].append(row["ANS_ORDER"])
    if max_ans_dict[row["題目序號"]]<row["ANS_ORDER"]:
        max_ans_dict[row["題目序號"]] = row["ANS_ORDER"]


In [11]:
for k1 in attr_dict.keys():
    for k2 in attr_dict[k1].keys():
        attr_dict[k1][k2] = list(set(attr_dict[k1][k2]))

In [14]:
# 問卷答案的字典
attr_dict

defaultdict(<function __main__.<lambda>()>,
            {261909: defaultdict(list,
                         {'A0': [1],
                          'A01': [8],
                          'A02': [1],
                          'A03': [1, 2],
                          'A04': [4],
                          'A05': [3],
                          'A06': [1],
                          'A07': [1],
                          'A08': [1, 3, 5],
                          'A09': [2, 3],
                          'A10': [2],
                          'A11': [1],
                          'A12': [1],
                          'A13': [1],
                          'A14': [4],
                          'B01': [3],
                          'B02': [1, 2, 3, 4, 5],
                          'B03': [3, 4, 5],
                          'B04': [5],
                          'B05': [4],
                          'B06': [4],
                          'B07': [4]}),
             269963: defaultdict(list,
           

#### 每個答案做成一個字 ， 每個 user 當一篇文章，計算答案的共現性


In [32]:
doc_word_list = []
for uid in attr_dict.keys():
    single_word_list = []
    for aid in attr_dict[uid].keys():
        single_word_list += [aid + "_" + str(ans) for ans in attr_dict[uid][aid]]
    doc_word_list.append(single_word_list)

In [37]:
names = list(set([w for l in doc_word_list for w in l]))
occurrences = OrderedDict((name, OrderedDict((name, 0) for name in names)) for name in names)
 
# Find the co-occurrences:
for l in doc_word_list:
    for i in range(len(l)):
        for item in l[:i] + l[i + 1:]:
            occurrences[l[i]][item] += 1
            
co_occur = pd.DataFrame.from_dict(occurrences)

In [41]:
co_occur.to_csv('./answer_cooccurance_matrix.csv')

### query and ranking 

In [149]:
query = "A01_2"
print("Ranking")
co_occur[query].sort_values(ascending=False)

Ranking


A0_1      119
A07_1     119
A03_1     119
A03_2     119
A14_4     118
A06_1     115
A09_1     112
B02_1     109
B03_4     108
A13_1     107
B02_4     102
A12_1      93
B03_5      92
A08_5      88
B02_3      86
A08_1      83
B02_5      83
B05_4      72
A05_2      69
A10_3      67
B01_5      64
A08_3      63
A09_3      61
B03_3      61
B04_5      60
B02_2      59
A04_5      57
A11_2      53
B07_5      50
B06_5      46
         ... 
A01_12      0
A01_13      0
A01_14      0
A01_2       0
A01_20      0
A01_18      0
A01_19      0
A01_23      0
A01_22      0
A01_5       0
A01_21      0
A01_4       0
B05_1       0
A01_6       0
A11_5       0
A08_7       0
A09_9       0
A0_3        0
A06_5       0
A06_4       0
A06_2       0
A14_2       0
A01_7       0
A14_3       0
B01_1       0
A04_1       0
B01_2       0
A01_9       0
A01_8       0
A01_1       0
Name: A01_2, Length: 117, dtype: int64

### t-testing

In [120]:
doc_word_list

[['A0_1',
  'A01_8',
  'A02_1',
  'A03_1',
  'A03_2',
  'A04_4',
  'A05_3',
  'A06_1',
  'A07_1',
  'A08_1',
  'A08_3',
  'A08_5',
  'A09_2',
  'A09_3',
  'A10_2',
  'A11_1',
  'A12_1',
  'A13_1',
  'A14_4',
  'B01_3',
  'B02_1',
  'B02_2',
  'B02_3',
  'B02_4',
  'B02_5',
  'B03_3',
  'B03_4',
  'B03_5',
  'B04_5',
  'B05_4',
  'B06_4',
  'B07_4'],
 ['A0_1',
  'A01_6',
  'A02_1',
  'A03_1',
  'A03_2',
  'A04_4',
  'A05_2',
  'A06_1',
  'A07_1',
  'A08_3',
  'A08_4',
  'A08_5',
  'A09_1',
  'A09_3',
  'A10_3',
  'A11_2',
  'A12_1',
  'A13_1',
  'A14_4',
  'B01_5',
  'B02_4',
  'B02_5',
  'B03_4',
  'B03_5',
  'B04_3',
  'B05_3',
  'B06_2',
  'B07_1'],
 ['A0_1',
  'A01_3',
  'A02_1',
  'A03_1',
  'A03_2',
  'A04_4',
  'A05_2',
  'A06_1',
  'A07_1',
  'A08_1',
  'A08_4',
  'A08_5',
  'A09_1',
  'A10_2',
  'A11_1',
  'A12_1',
  'A13_1',
  'A14_4',
  'B01_5',
  'B02_1',
  'B02_2',
  'B02_3',
  'B02_4',
  'B02_5',
  'B03_1',
  'B03_2',
  'B03_3',
  'B03_4',
  'B03_5',
  'B04_3',
  'B05_3',


In [128]:
np.unique(total_words, return_counts=True)

array([ 220,    3,   27,   38,   26,   16,    4,    1,  119,    4,  177,
         30,   17,   69,  178,  309,  265,   99,   73,  208,   11,  545,
        375,  150,  292,  101,   73,  356, 1884, 1884,   51,  192,  362,
        836,  443,  434, 1067,  387, 1746,   37,   16,    5,    4,   81,
       1882, 1235,  151,  941,  362, 1263,  150,    5, 1409,  304, 1064,
        131,   18,   25,  126,  119,   20, 1882,    3,  192,  589,  939,
        141,   36,  713,  861,  300,   15,    8, 1468,  318,  106, 1695,
        194,   34,   22,   20, 1806,   40,   85,  322,  709,  729, 1625,
        886, 1164, 1567, 1210,  285,  494,  871, 1535, 1315,   21,  171,
        469,  254,  982,    4,    6,  734, 1099,   51,   10,  154,  487,
        590,  656,   87,  232,  376,  445,  758], dtype=int64)

In [133]:
words_p_dict["A0_1"]

0.9642151887053613

In [134]:
words_p_dict["A0_1"]

0.9642151887053613

In [138]:
total_words = [word for doc in doc_word_list for word in doc]
total_words_count = len(total_words)
words_p_dict = {}
uniq = np.unique(total_words, return_counts=True)
for word, count in zip(uniq[0], uniq[1]):
    words_p_dict[word] = count/total_words_count
    
co_occur_prob = co_occur/total_words_count

In [155]:
t_test_dict = OrderedDict((name, OrderedDict((name, 0) for name in names)) for name in names)
significant_dict = defaultdict(list)
for name1 in names:
    for name2 in names:
        u = words_p_dict[name1]*words_p_dict[name2]
        if name1 == name2:
            x_bar = 1
        else:
            x_bar = co_occur_prob[name1][name2]
        if x_bar == 0.:
            value = 0
        else: 
            value = (x_bar-u)/(x_bar/total_words_count)**0.5
        t_test_dict[name1][name2] = value
        if value> 2.576:
            significant_dict[name1].append(name2)

t_test = pd.DataFrame.from_dict(t_test_dict)

In [156]:
significant_dict

defaultdict(list,
            {'B02_2': ['B02_2',
              'B07_5',
              'A02_4',
              'A01_22',
              'A06_2',
              'A11_1',
              'A02_6',
              'A12_2',
              'A08_4',
              'A04_1',
              'A03_1',
              'A05_2',
              'A07_1',
              'A05_1',
              'B01_5',
              'A02_2',
              'A14_2',
              'A08_1',
              'A14_4',
              'B07_4',
              'B03_4',
              'A10_3',
              'A09_4',
              'B03_1',
              'B04_5',
              'B02_3',
              'B01_1',
              'A01_6',
              'A05_3',
              'A04_3',
              'A01_2',
              'A04_4',
              'B06_5',
              'A08_5',
              'A01_4',
              'A01_21',
              'B05_5',
              'A09_5',
              'A13_1',
              'A10_1',
              'A01_13',
              'A08_6',
    

In [157]:
t_test["A01_2"].sort_values(ascending=False)

A01_2     232.775594
A0_1       10.529821
A07_1      10.529821
A03_1      10.529419
A03_2      10.529419
A14_4      10.497653
A06_1      10.366234
A09_1      10.290610
B02_1      10.098478
B03_4      10.067917
A13_1       9.984210
B02_4       9.758754
A12_1       9.309338
B03_5       9.290571
A08_5       9.085146
B02_3       8.997960
B02_5       8.818749
A08_1       8.812722
B05_4       8.200836
A05_2       8.024521
A10_3       7.933413
B01_5       7.799873
A08_3       7.676886
B03_3       7.565331
A09_3       7.511061
B04_5       7.467545
B02_2       7.427822
A04_5       7.420970
A11_2       7.020373
B07_5       6.835643
             ...    
A01_23      0.000000
A01_22      0.000000
A01_21      0.000000
A01_20      0.000000
A01_19      0.000000
A01_5       0.000000
A01_18      0.000000
A01_17      0.000000
A01_14      0.000000
A01_13      0.000000
A01_12      0.000000
A01_10      0.000000
A01_4       0.000000
A01_6       0.000000
A14_2       0.000000
A01_7       0.000000
A01_8       0

In [159]:
t_test.to_csv('./t_test.csv')

In [6]:
question_list = list(max_ans_dict.keys())

In [ ]:
# create feature matrix (one-hot encoding)
feature_matrix = []
id_list = []
for user in attr_dict.keys():
    single_user_feature = []
    for q in question_list:
        ans = np.array(attr_dict[user][q])-1
        one_hot = np.zeros(max_ans_dict[q])
        if len(ans)!=0:
            one_hot[ans] = 1
        single_user_feature.append(one_hot)
    merge_feature = np.hstack(single_user_feature)
    feature_matrix.append(merge_feature)
    id_list.append(user)
feature_matrix = np.vstack(feature_matrix)

In [19]:
# 客戶之風險等級
risk_dist = risk_qa[["客戶編號","風險等級"]].drop_duplicates()["風險等級"]

In [29]:
np.unique(risk_dist, return_counts=True)

(array([2, 4, 5], dtype=int64), array([   2,  101, 1786], dtype=int64))

In [ ]:
feature_matrix.shape

In [ ]:
X = feature_matrix
X_embedded = TSNE(n_components=2, perplexity=80).fit_transform(X)
X_embedded = PCA(n_components=2).fit_transform(X)

X_embedded.shape

In [ ]:
plt.figure(figsize=(16,9))
# cm = plt.cm.get_cmap("tab20", 18)
sc = plt.scatter(X_embedded[:,0], X_embedded[:,1])
plt.show()

# 實際投資行為

In [ ]:
invest = pd.read_excel("invest_data.xlsx")

In [ ]:
invest["weight"] = invest["風險等級"].apply(lambda x:int(x[-1]))
invest["weight_amount"] = invest["weight"]*invest["金額"]

In [ ]:
customer = list(set(invest["客戶"]))
customer_count = invest.groupby('客戶',as_index=False, sort=False).count()
customer_total = invest.groupby('客戶',as_index=False, sort=False).sum()
customer_total.head()

In [ ]:
customer_total["weight_level"] = customer_total["weight_amount"]/customer_total["金額"]

In [ ]:
customer_total.head()

In [ ]:
plt.hist(customer_total["金額"], bins=50)
# plt.xlim(0,50000)
plt.title("investment")
plt.show()

In [ ]:
sum(customer_money["金額"]<400000)/len(customer_money)

In [ ]:
customer_money.loc[customer_money["客戶"]==c, "風險等級"].values[0]

In [ ]:
vis_vec = []
amount = []
risk_level = []
for c in customer:
    money = customer_total.loc[customer_total["客戶"]==c, "金額"].values[0]
    level = customer_total.loc[customer_total["客戶"]==c, "weight_level"].values[0]
    if money<100000:
        vis_vec.append(X_embedded[id_list.index(c)])
        amount.append(money)
        risk_level.append(level)
vis_vec = np.vstack(vis_vec)

In [ ]:
plt.hist(risk_level,bins=40)
plt.title("weighted risk level")
plt.show()

In [ ]:
amount = (np.array(amount)-min(amount))/(max(amount)-min(amount))

In [ ]:
cmap = sns.cubehelix_palette(as_cmap=True)

f, ax = plt.subplots(figsize=(16,9))
points = ax.scatter(vis_vec[:,0], vis_vec[:,1], c=risk_level, s=50, cmap=cmap)
f.colorbar(points)
plt.title("risk level")
plt.show()

In [ ]:
cmap = sns.cubehelix_palette(as_cmap=True)

f, ax = plt.subplots(figsize=(16,9))
points = ax.scatter(vis_vec[:,0], vis_vec[:,1], c=amount, s=50, cmap=cmap)
f.colorbar(points)
plt.title("investment amount")
plt.show()

# 產品名稱

In [ ]:
product_corpus = "".join(list(invest["基金簡稱"]))

In [ ]:
jieba.set_dictionary('C:/Users/thtang/Desktop/dict.txt.big.txt')

In [ ]:
product_words = jieba.lcut(product_corpus)

In [ ]:
" ".join(product_words).replace("基金","").replace("野村","")

In [ ]:
freq = nltk.FreqDist(product_words)

In [ ]:
freq.most_common()